In [3]:
import pandas as pd
import plotly.express as px
from google.oauth2 import service_account
from google.cloud import bigquery
import streamlit as st


In [4]:
# Create API client.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
client = bigquery.Client(credentials=credentials)

# Perform query.
# Uses st.cache to only rerun when the query changes or after 10 min.
@st.cache(ttl=600)
def run_query(query):
    query_job = client.query(query)
    rows_raw = query_job.result()
    # Convert to list of dicts. Required for st.cache to hash the return value.
    rows = [dict(row) for row in rows_raw]
    return rows

rows = run_query("SELECT * FROM psycho_research.df_demographics")


2022-01-26 17:34:12.298 
  command:

    streamlit run /Users/gustavodiez/Library/Caches/pypoetry/virtualenvs/streamlit-dash-9GqNcUWI-py3.9/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/gustavodiez/PycharmProjects/streamlit_dash/notebooks/.streamlit/secrets.toml'